In [13]:
import pandas as pd
import requests
import bs4 as beautifulsoup
from bs4 import BeautifulSoup
#https://www.nbadraft.net/search-players/ Website we will iterate through

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

base_url = 'http://www.nbadraft.net'


In [9]:
def ScrapeProfile():
    pass

url_list = []

for i in range(215):
    url_link = base_url + "/search-players/"
    if i == 0:
        url_list.append(url_link)
    else: 
        url_link = url_link + "?offset="+str(i)
        url_list.append(url_link)
    
#https://www.nbadraft.net/search-players/ ~ first page
#https://www.nbadraft.net/search-players/?offset=210 ~ all other pages

player_links = []

In [11]:
url_list[:5]

['http://www.nbadraft.net/search-players/',
 'http://www.nbadraft.net/search-players/?offset=1',
 'http://www.nbadraft.net/search-players/?offset=2',
 'http://www.nbadraft.net/search-players/?offset=3',
 'http://www.nbadraft.net/search-players/?offset=4']

In [14]:
soup = BeautifulSoup(url_list[0], "html.parser")

for 

/home/truman/anaconda3/envs/ufc/lib/python3.6/site-packages/bs4/__init__.py:389: UserWarning: "http://www.nbadraft.net/search-players/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [ ]:
for pages in url_list:
    for player_link in player_links:
        ScrapeProfile()